In [1]:
from datetime import datetime
import sys
def now():return mktime(datetime.now().timetuple())
def now_s():
    s = datetime.now().isoformat()
    return s[:s.index('.')]
now_s()

def timer(slient_mode = False):
    pf = print
    def empty_func(*args):
        pass
    pf = empty_func if slient_mode else print
    
    def split_by_time(*args):
        pf("-"*60 , now_s(), *args)
    
    split_by_time()
    t1 = now()
    yield t1, 0
    
    while True:
        t2 = now()
        split_by_time("time costs %2d secs"%(t2 - t1))
        yield t2, t2 - t1

In [2]:
#export
import os

IN_TRAVIS=(os.getenv('TRAVIS', False) == 'true')

In [3]:
#export
#https://stackoverflow.com/questions/15411967/how-can-i-check-if-code-is-executed-in-the-ipython-notebook
def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError: 
        return False      # Probably standard Python interpreter
IN_JUPYTER = isnotebook()

In [4]:
#export
import tqdm as _tqdm
def _simple_tqdm(g):
    """
    for travis
    """
    try:
        l = len(g)
    except TypeError:
        l = '?'
    for i,x in enumerate(g):
        print(f"({i}/{l})", end='')
        yield x
        
if IN_JUPYTER:
    tqdm = _tqdm.tqdm_notebook
elif IN_TRAVIS:
    tqdm = _simple_tqdm
else :
    tqdm = _tqdm.tqdm

## notebook

In [5]:
# export
def ex_command(code):
    ip = get_ipython()
    ip.run_cell(code)

In [6]:
#export

# use those only in jupyter
from IPython.display import display, Javascript
import time

def save_notebook(): display(Javascript('IPython.notebook.save_checkpoint();'))

def restart_kernel():display(Javascript('IPython.notebook.kernel.restart();'))
    
def save_and_export_notebook(name):
    
    assert IN_JUPYTER
    
    save_notebook()
    NOTEBOOK_EXTEND_NAME='.ipynb'
    if NOTEBOOK_EXTEND_NAME not in name:
        name += NOTEBOOK_EXTEND_NAME
    time.sleep(1)
    
    # support import complie
    ex_command(f'!python notebook2script.py {name}')
    ex_command(f'!python notebook2test_script.py {name}')
    
    save_notebook() # for exitting
    

## linux command

In [7]:
def gpu_usage():
    ex_command('nvidia-smi')

## asyncio

In [8]:
#export

# https://docs.python.org/3/library/asyncio-subprocess.html
import asyncio
import sys
import datetime
import time

# https://stackoverflow.com/questions/44633458/why-am-i-getting-notimplementederror-with-async-and-await-on-windows
# asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy()) 

async def async_run(run_args=[sys.executable, '-c', "import time;print('ing',  flush=True);time.sleep(2);print('done')",]
                   , wait_child_sec=3):

    # Create the subprocess; redirect the standard output
    # into a pipe.
    proc = await asyncio.create_subprocess_exec(
        *run_args,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    await asyncio.wait_for(proc.wait(), timeout=wait_child_sec)
    
    line=''
    
    data = await proc.stdout.read()
    line += data.decode('ascii').rstrip()
    
    line += '\n'
    
    data = await proc.stderr.read()
    line += data.decode('ascii').rstrip()
    
    if line: print(line)
    

## output

In [ ]:
save_and_export_notebook('common')

<IPython.core.display.Javascript object>

Converted common.ipynb to ..\exp\common.py
Converted common.ipynb to ..\test\test_common.py


In [ ]:
from pathlib import Path

In [ ]:
Path('a')/'..'/'b'